In [286]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import os
import prepare_flightlog as fl


In [287]:
base_dir = os.path.dirname(os.getcwd())
flightlogs_dir = os.path.join(base_dir, "data", "flightlogs")
files = [f for f in os.listdir(flightlogs_dir)
         if f.endswith(".igc") or f.endswith(".IGC")]
print(files)

['2025-05-10-XCT-YCA-01.igc', '2025-05-10-XCT-FBA-02.igc', '2025-05-10-XNA-170300-01.igc']


In [288]:
file_no = 1
file_path = os.path.join(flightlogs_dir, files[file_no])
print(f"Reading {file_path}")
df = fl.prepare_data(file_path)
df.head()

Reading /Users/serkan/Documents/Projeler/Paragliding_Flight_Analyses/data/flightlogs/2025-05-10-XCT-FBA-02.igc


,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,...,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,/Users/serkan/Documents/Projeler/Paragliding_F...,2025-05-10 11:34:59,Ferkan Bayram,37.326450,37.177933,1087,1106,37.326433,37.178017,29.543230,...,284,14.0,23.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0
1,/Users/serkan/Documents/Projeler/Paragliding_F...,2025-05-10 11:35:00,Ferkan Bayram,37.326483,37.177850,1087,1107,37.326450,37.177933,37.113869,...,296,12.0,24.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0
2,/Users/serkan/Documents/Projeler/Paragliding_F...,2025-05-10 11:35:01,Ferkan Bayram,37.326500,37.177767,1087,1107,37.326483,37.177850,44.660920,...,284,12.0,25.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0
3,/Users/serkan/Documents/Projeler/Paragliding_F...,2025-05-10 11:35:02,Ferkan Bayram,37.326500,37.177683,1089,1107,37.326500,37.177767,51.997589,...,270,14.0,26.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0
4,/Users/serkan/Documents/Projeler/Paragliding_F...,2025-05-10 11:35:03,Ferkan Bayram,37.326500,37.177617,1090,1107,37.326500,37.177683,57.875925,...,270,0.0,27.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0


In [289]:
print(df["elapsed_time"].iloc[-1])

7908.0


In [290]:
def calc_zscore(df) -> pd.Series:
    mean = df.mean()
    std = df.std()
    result = (df - mean) / std
    result = np.where(abs(result) > 3, True, False)
    return result


df = df[['latitude', 'longitude',
         'gps_altitude_m', 'distance_m', 'speed_km/s',
         'climb_m', 'climb_m(delta)', 'climb_rate_m/s',
         'glide_ratio', 'bearing', 'delta_bearing',
         'elapsed_time', 'delta_time', 'temp',
         'pressure', 'humidity', 'dew_point',
         'wind_speed', 'wind_deg']]
df = df[df['gps_altitude_m'] > 0]

df_cleaned = df.copy()
zsore_columns = ['gps_altitude_m', 'distance_m', 'speed_km/s',
                 'climb_m', 'climb_m(delta)', 'climb_rate_m/s',
                 'glide_ratio', 'delta_time']

for col in zsore_columns:
    df_cleaned[col + "_zscore"] = calc_zscore(df_cleaned[col].copy())
    df_cleaned = df_cleaned[df_cleaned[col + "_zscore"] == False]
    df_cleaned.drop(columns=[col + "_zscore"], inplace=True)

df = df_cleaned
df.describe()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
count,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.0,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000,7830.000000
mean,37.325535,37.453661,1795.562835,10.571645,38.057923,-0.063857,-1.442401,-0.063857,5.540227,123.623499,8.334738,3965.242273,1.0,30.242990,1010.837931,20.080460,4.842810,1.621894,267.076501
std,0.066237,0.175982,589.071062,3.424742,12.329070,1.929994,35.177871,1.929994,4.534655,90.554482,8.705098,2282.611891,0.0,1.303713,0.720601,1.178603,0.368068,0.576118,21.336447
min,37.235683,37.176833,546.000000,1.475151,5.310545,-5.000000,-94.000000,-5.000000,0.000000,0.000000,0.000000,23.000000,1.0,27.810000,1010.000000,19.000000,4.170000,0.620000,222.000000
25%,37.267367,37.297700,1289.000000,7.966952,28.681028,-1.000000,-27.000000,-1.000000,2.341475,52.000000,2.000000,1981.250000,1.0,29.370000,1010.000000,19.000000,4.790000,1.180000,243.000000
50%,37.307925,37.426033,1765.500000,10.989996,39.563985,0.000000,-8.000000,0.000000,4.735549,89.000000,7.000000,3968.500000,1.0,30.610000,1011.000000,20.000000,4.920000,1.590000,275.000000
75%,37.395579,37.627375,2258.000000,13.277466,47.798877,1.000000,29.000000,1.000000,7.966986,180.000000,11.000000,5950.750000,1.0,31.590000,1011.000000,21.000000,5.180000,1.910000,282.000000
max,37.437283,37.718417,3121.000000,20.596521,74.147476,5.000000,92.000000,5.000000,18.936535,352.000000,91.000000,7908.000000,1.0,31.700000,1012.000000,22.000000,5.710000,2.380000,298.000000


In [291]:
# --- 1. Load saved scaler and Transformer model ---
scaler_path = os.path.join('models', 'scaler.pkl')
model_path  = os.path.join('models', 'transformer_tunned_model.keras')

with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)

model = tf.keras.models.load_model(model_path)
print(f"Loaded Transformer model from '{model_path}'")


# Define feature columns (everything except the target)
feature_cols = [c for c in df.columns if c != 'climb_rate_m/s']


Loaded Transformer model from 'models/transformer_tunned_model.keras'


In [292]:
# --- 3. Scale features ---
X_new = df[feature_cols].values.astype(np.float32)
X_new_scaled = scaler.transform(X_new)

# --- 4. Create sliding‐window dataset for inference ---
T = 10        # same window length as training
batch_size = 32

predict_ds = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=X_new_scaled,
    targets=None,
    sequence_length=T,
    sequence_stride=1,
    shuffle=False,
    batch_size=batch_size
)

In [293]:
# --- 5. Run predictions ---
preds = model.predict(predict_ds).flatten()

# --- 6. Align predictions with original data indices or timestamps ---
# If your DataFrame has a timestamp column, you can use that instead.
# number of windows = N – T + 1
preds = model.predict(predict_ds).flatten()
# Align preds to df_new by taking index[T-1:]
indices = df.index[T-1:]
# Just in case, truncate to the same length:
indices = indices[: len(preds)]

indices = df.index[T-1 : T-1 + len(preds)]
df.loc[indices, 'predicted_climb_rate_m/s'] = preds

245/245 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [294]:
df = df[df['predicted_climb_rate_m/s'].notna()]
df = df[df['climb_rate_m/s'] > 0]
df.head()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg,predicted_climb_rate_m/s
15,37.325850,37.177250,1081,11.484864,41.345511,1.0,-7.0,1.0,11.484864,194,5.0,38.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0,1.150311
19,37.325567,37.177200,1085,7.398917,26.636101,1.0,-7.0,1.0,7.398917,180,11.0,42.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0,1.098081
21,37.325467,37.177183,1086,3.699458,13.318050,1.0,-6.0,1.0,3.699458,180,11.0,44.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0,1.170704
22,37.325417,37.177167,1089,5.742434,20.672762,1.0,-5.0,1.0,5.742434,194,14.0,45.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0,1.169335
23,37.325367,37.177183,1090,5.742434,20.672763,2.0,-3.0,2.0,2.871217,165,29.0,46.0,1.0,27.81,1012.0,22.0,4.17,1.59,282.0,1.169443


In [295]:
y_true = df['climb_rate_m/s'].values
y_pred = df['predicted_climb_rate_m/s'].values

# 2) Pearson korelasyon (−1…1 → map 0…1)
pearson_r = np.corrcoef(y_true, y_pred)[0,1]
pearson_sim = (pearson_r + 1) / 2
pearson_percent = pearson_sim * 100
print(f"Pearson similarity: {pearson_percent:.2f}%")

Pearson similarity: 70.09%


In [296]:
# Orijinal y_true, y_pred
y_true_centered = y_true - np.mean(y_true)
y_pred_centered = y_pred - np.mean(y_pred)

# Normalize
y_true_n = y_true_centered / np.linalg.norm(y_true_centered)
y_pred_n = y_pred_centered / np.linalg.norm(y_pred_centered)

cos_sim2 = cosine_similarity(
    y_true_n.reshape(1,-1),
    y_pred_n.reshape(1,-1)
)[0,0]
print(f"Centered Cosine: {cos_sim2*100:.2f}%")

Centered Cosine: 40.17%


In [297]:
# 1) Gerçek ve tahmin verilerini oku
y_true = df['climb_rate_m/s'].values
y_pred = df['predicted_climb_rate_m/s'].values

# 2) Parametreler
E_max = 3.0    # 2 m/s üzerindeki hataları sıfır kabul et

# 3) Zaman adımı puanları
errors = y_true - y_pred
score_t = np.clip(1 - (errors / E_max), 0, 1) * 100

# Her adım en fazla 100 olmalı
flight_score = score_t.mean()

print(f"Bu uçuş için performans skoru: {flight_score:.2f}/100")

Bu uçuş için performans skoru: 87.43/100


In [298]:
def calc_score(row):
    error =  row['predicted_climb_rate_m/s'] - row['climb_rate_m/s']
    return (1-(np.clip(error, 0, E_max)/E_max))*100


df["score"] = df.apply(calc_score, axis=1)
print(df["score"].mean())



df[["climb_rate_m/s", "predicted_climb_rate_m/s", "score"]]

85.7510358503021


,climb_rate_m/s,predicted_climb_rate_m/s,score
15,1.0,1.150311,94.989645
19,1.0,1.098081,96.730649
21,1.0,1.170704,94.309851
22,1.0,1.169335,94.355504
23,2.0,1.169443,100.000000
...,...,...,...
6637,1.0,1.245903,91.803241
6642,1.0,1.158161,94.727961
6666,1.0,1.252213,91.592908
6704,1.0,1.159656,94.678140
